In [1]:
import pandas as pd


In this data question you will work with service request data related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system.

Your goal is to practice working with dates, times, and time-based aggregation in pandas while exploring patterns in missed pickups. You’ll also calculate fines based on missed pickups within a rolling time window.

Step 1: Data Preparation



In [5]:

trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
trash.head(2)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470


In [6]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


#### 1. Convert any date columns to datetime using pd.to_datetime().

In [8]:
trash["Date Opened"] = pd.to_datetime(trash['Date Opened'], format = "%m/%d/%Y")

In [9]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Request Number    20226 non-null  int64         
 1   Date Opened       20226 non-null  datetime64[ns]
 2   Request           20226 non-null  object        
 3   Description       20195 non-null  object        
 4   Incident Address  20217 non-null  object        
 5   Zip Code          20151 non-null  float64       
 6   Trash Hauler      19325 non-null  object        
 7   Trash Route       19279 non-null  object        
 8   Council District  20177 non-null  float64       
 9   State Plan X      20198 non-null  float64       
 10  State Plan Y      20198 non-null  float64       
 11  LONGITUDE         20198 non-null  float64       
 12  LATITUDE          20198 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 2.0+ MB


#### 2. Create new columns for:

#### year, month, day

#### day_of_week (or weekday name)

In [13]:
trash["Year"] = trash["Date Opened"].dt.year
trash["Month"] = trash["Date Opened"].dt.month
trash["Day"] = trash["Date Opened"].dt.day
trash["Day_of_Week"] = trash["Date Opened"].dt.day_name()

In [14]:
trash.head(2)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Year,Month,Day,Day_of_Week
0,25270,2017-11-01,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292,2017,11,1,Wednesday
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470,2017,11,1,Wednesday


In [15]:
trash["Request"].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [16]:
trash["Damage"] = trash["Request"] == 'Damage to Property'

In [17]:
trash_damage = trash[trash.Request.str.contains('Damage to Property')]
trash_damage.head(2)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Year,Month,Day,Day_of_Week,Damage
6,25325,2017-11-01,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,RED RIVER,3303,34.0,1726300.965,637078.4022,-86.819640,36.080729,2017,11,1,Wednesday,True
173,26971,2017-11-08,Damage to Property,truck is cutting into yard and damaging lawn,717 garrison dr,37207.0,RED RIVER,3205,2.0,1727317.062,688538.0878,-86.817657,36.222107,2017,11,8,Wednesday,True


In [18]:
trash

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Year,Month,Day,Day_of_Week,Damage
0,25270,2017-11-01,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292,2017,11,1,Wednesday,False
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470,2017,11,1,Wednesday,False
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923,2017,11,1,Wednesday,False
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652,2017,11,1,Wednesday,False
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861,2017,11,1,Wednesday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130,2019,11,1,Friday,False
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016,-86.741242,36.169482,2019,11,1,Friday,False
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643,2019,11,1,Friday,False
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637,2019,11,1,Friday,False


In [19]:
trash_missed = trash[trash["Request"].isin(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup', 'Trash Collection Complaint'])]

In [20]:
trash_missed["Request"].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint'], dtype=object)

In [57]:
trash_missed["Description"][1000]

'MISSED PICK UP-Caller states this is an office building with 3 trash carts- caller states some items in cart are lose but that is the way the trash is normally picked up.'

In [22]:
trash_backdoor = trash[trash["Request"].isin(['Trash - Backdoor'])]

In [41]:
trash_backdoor

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Year,Month,Day,Day_of_Week,Damage
0,25270,2017-11-01,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292,2017,11,1,Wednesday,False
55,25692,2017-11-02,Trash - Backdoor,Backdoor/miss for last Friday. Does not unders...,3041 Fieldstone Dr,37013.0,RED RIVER,3504,29.0,1785677.159,633139.2463,-86.618624,36.071107,2017,11,2,Thursday,False
63,25775,2017-11-02,Trash - Backdoor,Missed trash pickup said has been picking up b...,4625 long br,37013.0,RED RIVER,4505,28.0,1770350.416,634143.8718,-86.670507,36.073590,2017,11,2,Thursday,False
71,25802,2017-11-02,Trash - Backdoor,Missed.,4828 Cimarron Way,37013.0,RED RIVER,3404,30.0,1766186.564,629785.6029,-86.684493,36.061538,2017,11,2,Thursday,False
78,25858,2017-11-03,Trash - Backdoor,Missed- she said her grandson was over yesterd...,2005 Lebanon Pike,37210.0,RED RIVER,1504,15.0,1760561.763,665276.0012,-86.704387,36.158919,2017,11,3,Friday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20165,266605,2019-10-31,Trash - Backdoor,HAS MISSED BACK DOOR TRASH PICK UP AGAIN/ ALSO...,"4305 Granny White Pike, Nashville, TN 37204, U...",37204.0,RED RIVER,3302,25.0,1731857.800,644223.2015,-86.801034,36.100482,2019,10,31,Thursday,False
20167,266622,2019-10-31,Trash - Backdoor,MISSED BACKDOOR PICK UP HERE,"4226 Eatons Creek Rd, Nashville, TN 37218, Uni...",37218.0,RED RIVER,3203,1.0,1713726.602,685432.7984,-86.863636,36.213257,2019,10,31,Thursday,False
20202,266912,2019-11-01,Trash - Backdoor,"missed multiple times, please pick up","3110 Lebanon Rd, Lebanon, TN 37087, United States",37214.0,RED RIVER,1505,14.0,1778640.200,673206.4002,-86.643325,36.181045,2019,11,1,Friday,False
20205,266944,2019-11-01,Trash - Backdoor,backdoor miss/ again Thursday,"812 Horner Ave, Nashville, TN 37204, United St...",37204.0,METRO,9404,17.0,1738948.631,652136.9083,-86.777245,36.122378,2019,11,1,Friday,False
